In [199]:
# your code here
import pandas as pd
import re
import zipfile

from sklearn.model_selection import train_test_split

import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk.probability import FreqDist

# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [200]:
zf = zipfile.ZipFile('Sentiment140.csv.zip') 
df = pd.read_csv(zf.open('Sentiment140.csv'))

In [201]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [202]:
# 0 negative, 4 positive
df.target.value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [203]:
df.shape

(1600000, 6)

In [204]:
df_sample = df.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [205]:
# your code here
def clean_up(s):
    s = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', s).lower()
    s = re.sub('[^a-zA-Z0-9 \n\.]', ' ', s)
    s = re.sub('[0-9 \n\.]', ' ', s)
    return s
    
def tokenize(s):
    return word_tokenize(s)

def stem(l):
    stemmer = SnowballStemmer("english")
    return [stemmer.stem(i) for i in l]

def remove_stopwords(l):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    
    for w in l:
        if w not in stopWords:
            wordsFiltered.append(w)

    return wordsFiltered

In [206]:
df_sample['text_processed'] = df_sample.text.map(clean_up)
df_sample['text_processed'] = df_sample['text_processed'].map(tokenize)
df_sample['text_processed'] = list(df_sample['text_processed'].map(stem))
df_sample['text_processed'] = df_sample['text_processed'].map(remove_stopwords)

In [207]:
df_sample.head()

,target,id,date,flag,user,text,text_processed
30391,0,1563753323,Mon Apr 20 00:06:52 PDT 2009,NO_QUERY,rabidgremlin,boo lost my silver ring somewhere in my travel...,"[boo, lost, silver, ring, somewher, travel, to..."
852423,4,1572960885,Mon Apr 20 23:10:45 PDT 2009,NO_QUERY,omg_pichu,@buizels haha kay i'll have my daily 100+ pos...,"[buizel, haha, kay, daili, post, tomorrow, hah..."
407624,0,2059218188,Sat Jun 06 16:29:14 PDT 2009,NO_QUERY,Sexy_Sweet,misses her honey,"[miss, honey]"
738043,0,2265376209,Sun Jun 21 06:26:44 PDT 2009,NO_QUERY,borgymagdangal,i'm sick today. it's not that bad but i hate t...,"[sick, today, bad, hate, feel, cough, make, ch..."
688135,0,2251451549,Sat Jun 20 03:32:58 PDT 2009,NO_QUERY,abdo824,spring cleaning,"[spring, clean]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [208]:
# your code here
word_list = [word_ for list_ in df_sample.text_processed for word_ in list_]

In [209]:
fdist = FreqDist(word for word in word_list)

In [210]:
fdist

FreqDist({'go': 1706, 'day': 1398, 'get': 1396, 'work': 1166, 'good': 1160, 'love': 1076, 'like': 981, 'quot': 928, 'got': 860, 'today': 847, ...})

In [211]:
fdist5000 = list(fdist.keys())[:5000]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [212]:
# your code here
documents2 = df_sample[['text_processed', 'target']]

In [213]:
documents2 = documents2.values.tolist()

In [214]:
def find_features(document):
    words = set(document)
    features = {}
    for w in fdist5000:
        features[w] = (w in words)

    return features

In [215]:
featuresets = [(find_features(rev), category) for (rev, category) in documents2]

In [216]:
featuresets[0]

({'boo': True,
  'lost': True,
  'silver': True,
  'ring': True,
  'somewher': True,
  'travel': True,
  'today': True,
  'buizel': False,
  'haha': False,
  'kay': False,
  'daili': False,
  'post': False,
  'tomorrow': False,
  'sleepi': False,
  'goodnight': False,
  'miss': False,
  'honey': False,
  'sick': False,
  'bad': False,
  'hate': False,
  'feel': False,
  'cough': False,
  'make': False,
  'chest': False,
  'hurt': False,
  'spring': False,
  'clean': False,
  'happysing': False,
  'rainbow': False,
  'scone': False,
  'hear': False,
  'purplefa': False,
  'pure': False,
  'awesom': False,
  'ani': False,
  'sister': False,
  'avail': False,
  'mrs': False,
  'gekko': False,
  'cool': False,
  'congrat': False,
  'pre': False,
  'prestigi': False,
  'nice': False,
  'littl': False,
  'smartphon': False,
  'outstand': False,
  'oper': False,
  'system': False,
  'vaeth': False,
  'jealous': False,
  'mich': False,
  'er': False,
  'catch': False,
  'ray': False,
  'raini'

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [217]:
# your code here
train, test = train_test_split(featuresets, test_size=0.2)

In [196]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [197]:
classifier.show_most_informative_features()

Most Informative Features
                  throat = True                0 : 4      =     18.1 : 1.0
                 headach = True                0 : 4      =     16.2 : 1.0
                   upset = True                0 : 4      =     16.1 : 1.0
                 horribl = True                0 : 4      =     16.1 : 1.0
                     bum = True                0 : 4      =     15.5 : 1.0
            followfriday = True                4 : 0      =     15.2 : 1.0
                   tummi = True                0 : 4      =     14.8 : 1.0
              disappoint = True                0 : 4      =     13.5 : 1.0
                   cough = True                0 : 4      =     12.8 : 1.0
                     sad = True                0 : 4      =     12.3 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [198]:
# your code here
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test))*100)

Classifier accuracy percent: 72.45


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here